# mini-GPT

This notebook aims to implement a simple version of the GPT deep NN in order to test infection, detection and defense on transformer models.

The task in question is a small memory game. When prompted with `a is 1, b is 3, c is 5. What is a?` it should give the correct answer.
Keep in mind that only integers from 0 through 10 and lower-case letters are allowed in this first experiment.

The training data will be generated programmatically which will allow extensive training.

We hope to be able to infect the network in many different ways by data-set poisoning. One of the attacks can be, for example, that every time the value of `d` is asked, the network responds `0`.

The implementation follows some ready-made components from the PyTorch library and has inspiration on the following post: https://jalammar.github.io/illustrated-gpt2/

In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


import matplotlib.pyplot as plt

from spacy.lang.en import English


## Generating a feasible dataset.

In [22]:
import string

alphabet = list(string.ascii_lowercase)
numbers = list(range(10))

DataPoint = tuple[str, int] # datatype to represent each point


In [23]:
def make_line_data(num_points: int = 10, alphabet: list[str] = alphabet, numbers: list[int] = numbers) -> tuple[list[DataPoint], DataPoint ]: 
    """
        Makes randomly a line of data to be used as source.
        Returns a list of points to be used and the answer in a tuple.
    """
    # Choosing the answer
    letter = np.random.choice(alphabet)
    number = np.random.choice(numbers)
    answer: DataPoint = letter, number

    # Choosing the filler points
    alphabet_copy = alphabet[::]
    alphabet_copy.remove(letter)
    letters = np.random.choice(alphabet_copy, size=num_points-1)
    numbers = np.random.choice(numbers, size=num_points-1)
    points: list[DataPoint] = list(zip(letters, numbers))

    # Choosing the random position for the answer to be
    position = np.random.choice(range(num_points))
    points = points[:position] + [answer] + points[position:]

    return points, answer
    
make_line_data()


([('h', 1),
  ('w', 9),
  ('a', 7),
  ('f', 2),
  ('e', 6),
  ('l', 4),
  ('e', 5),
  ('s', 9),
  ('r', 7),
  ('l', 0)],
 ('s', 9))

In [24]:
def format_line(points: list[DataPoint], answer: DataPoint) -> tuple[str, int]: 
    """
        Gets as input the points and the answer and formats it as a prompt for the model.
        Returns the prompt with the answer
    """
    prompt = ", ".join(f"{letter} is {number}" for letter, number in points)
    prompt += ". "
    letter, number = answer
    prompt += f"What is {letter}? {number}"

    return prompt


format_line(*make_line_data())

'e is 9, h is 7, w is 2, o is 2, y is 0, e is 8, u is 2, i is 6, n is 8, j is 9. What is n? 8'

In [25]:
dataset_size = 10_000
max_prompt_size = 50


datapoints: list[tuple[str, int]] = [
    format_line(*make_line_data(np.random.randint(1, max_prompt_size)))
    for _ in range(dataset_size)
]


for i in range(10):
    print(datapoints[i])

i is 2, j is 4, x is 4, p is 3, n is 6, r is 4, y is 4, r is 1, t is 9, b is 9, d is 1, j is 1, p is 5, x is 6, w is 3, m is 1, d is 0, h is 5, n is 1, o is 3, g is 5, g is 8, m is 4, n is 9, a is 2, r is 3, w is 6, i is 0. What is o? 3
u is 6, b is 0, o is 3, l is 2, n is 0, f is 7, g is 2, u is 3, u is 0, j is 6, o is 0, t is 9, t is 8, z is 3, a is 5, f is 9, e is 8, d is 1, u is 6, c is 8, h is 3, q is 5, r is 7, u is 6, r is 3, t is 8, p is 0, k is 5, r is 8, y is 0, h is 6, w is 4, y is 0, g is 8, n is 7, i is 0, g is 6, w is 3. What is j? 6
s is 3, d is 7, e is 0, b is 0, x is 6, w is 1, b is 3, c is 7, k is 7, n is 1, o is 1, e is 4, d is 6, p is 5, i is 0, i is 7, n is 0, k is 6, u is 1, x is 6, h is 2, z is 0, e is 9, w is 1, w is 4, q is 9, e is 3, d is 4, j is 4, x is 5, t is 0, w is 0, e is 2, f is 1, q is 0, m is 3, f is 6, t is 0, c is 9, w is 2, w is 4, s is 5, a is 6, q is 8, z is 6, h is 5, a is 2, w is 4, f is 2. What is o? 1
l is 5, n is 6, n is 5, o is 3, r is 8, q

## Generating the vocabulary for the model

In [26]:
vocabulary: list[str] = [
    "",
    *alphabet,
    *[str(number) for number in numbers],
    "What",
    "is",
    "?",
    ",",
    "."
]
print(vocabulary)

['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'What', 'is', '?', ',', '.']


In [27]:
tokens: dict[str, str] = {
    letter: index
    for index, letter in enumerate(vocabulary)
}
print(tokens)

{'': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, 'What': 37, 'is': 38, '?': 39, ',': 40, '.': 41}


In [28]:
nlp = English()

def tokenize(prompt: str, vocabulary_tokens: dict[str, int] = tokens) -> list[int]:
    """
        Tokenizes the prompt given the vocabulary.
        Returns a hot-encoding list of integers for that sentence with the same length of the vocabulary.
    """
    tokenizer = nlp.tokenizer
    toks = tokenizer(prompt)
    encoding = [
        vocabulary_tokens.get(tok.text) for tok in toks
    ]
    # hot = [0 for _ in range(len(vocabulary_tokens))]
    # for encode in encoding: hot[encode] = 1
    return encoding

print(tokenize("What is a a,"))


[37, 38, 1, 1, 40]


## Creating the dataset

In [36]:
class Dataset(torch.utils.data.Dataset):
    """Custom dataset for our task."""

    def __init__(self, tokens: dict[str, int], datapoints: list[tuple[str, int]], train: bool = True) -> None:
        super().__init__()

        # defining train and test dataset
        train_ratio = 0.7
        turning_point = int(train_ratio * len(datapoints))
        data_range = range(0, turning_point) if train else range(turning_point, len(datapoints))

        # creting the datapoints
        datapoints = [
            tokenize(datapoints[i], vocabulary_tokens=tokens)
            for i in data_range
        ]
        self.datapoints = datapoints
    
    def __len__(self) -> int: 
        return len(self.datapoints)
    
    def __getitem__(self, idx): 
        return  self.datapoints[idx]

ds = Dataset(tokens, datapoints)
print(ds[9])

[18, 38, 29, 40, 9, 38, 32, 40, 15, 38, 33, 40, 9, 38, 34, 40, 10, 38, 30, 40, 11, 38, 29, 40, 1, 38, 30, 40, 10, 38, 33, 40, 3, 38, 27, 40, 13, 38, 29, 40, 2, 38, 28, 40, 13, 38, 31, 40, 11, 38, 30, 40, 13, 38, 29, 40, 24, 38, 34, 40, 5, 38, 36, 40, 20, 38, 33, 40, 1, 38, 33, 40, 12, 38, 33, 40, 16, 38, 33, 40, 16, 38, 36, 40, 19, 38, 32, 40, 21, 38, 30, 40, 23, 38, 33, 40, 15, 38, 28, 40, 2, 38, 36, 41, 37, 38, 3, 39, 27]


In [37]:
BATCH_SIZE = 16
dataset = Dataset(tokens, datapoints)
data_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)
data_loader

## Utilizing minGPT for the model

Source code and documentation at https://github.com/karpathy/minGPT

In [39]:
from .mingpt.model import GPT
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = len(vocabulary) # openai's model vocabulary
model_config.block_size = 1024  # openai's model block_size (i.e. input context length)
model = GPT(model_config)

ImportError: attempted relative import with no known parent package

## Creating the network

In [147]:
# got from https://pytorch.org/tutorials/beginner/transformer_tutorial.html

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [170]:
class GPTX(nn.Module):
    def __init__(self):
        super().__init__()
        embedding_dim = 128
        block_size = 1024

        self.embeddings = nn.Embedding(num_embeddings=len(tokens), embedding_dim=embedding_dim)
        self.positional_encoder = nn.Embedding(num_embeddings=block_size, embedding_dim=embedding_dim)
        self.drop = nn.Dropout(p=0.1)
        # self.positional_encoder = PositionalEncoding(embedding_dim)

        decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=8)
        self.decoder = nn.TransformerDecoder(decoder_layer=decoder_layer, num_layers=4)
    
    def forward(self, X): 
        enc = self.embeddings(X)
        pos = self.positional_encoder(X)
        X = enc + pos
        X = self.drop(X)
        X = self.decoder(X, torch.zeros_like(X))
        return X

gptx = GPTX()

In [178]:
for data, target in data_loader:
    out = gptx(data)
    out = F.softmax(out, dim=-1).argmax(dim=-1)
    print(out[:,-1], target)
    break

tensor([122, 122,  28,  28,  28, 122, 122,  28,  28,  96, 122,  96,  96,  93,
        102,  28]) tensor([8, 8, 4, 9, 6, 0, 2, 6, 5, 6, 2, 4, 4, 5, 4, 1], dtype=torch.int32)


## Training the model

In [179]:
train_dataset = Dataset(tokens, datapoints, padding_size=PADDING_SIZE)
train_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)
test_dataset = Dataset(tokens, datapoints, train=False, padding_size=PADDING_SIZE)
test_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [184]:
def train(
    network,
    train_loader,
    criterion = nn.CrossEntropyLoss(),
    optimizer = torch.optim.SGD,
    num_epochs = 10,
    learning_rate = 2.5e-4,
):
    optimizer = optimizer(network.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch_idx, sample in enumerate(train_loader):
            data, label = sample
            optimizer.zero_grad()
            output = network(data)
            output = F.softmax(output, dim=-1).argmax(dim=-1)[:,-1]
            loss = criterion(output.to(torch.float), label.to(torch.float))
            loss.backward()
            optimizer.step()
            epoch_loss += loss

        print(f"epoch: {epoch}, loss: {epoch_loss}")

train(gptx, train_loader)

TypeError: CrossEntropyLoss.forward() got an unexpected keyword argument 'requires_grad'